# Scanning Session Ingestion

## Setup

### Connect to the database

If you are don't have your login information, contact the administrator.

Using local config file (see [01_pipeline](./01_pipeline.ipynb)):

Check out the respective elements video: https://www.youtube.com/watch?v=DLYqnj40eAM


In [1]:
import os
# change to the upper level folder to detect dj_local_conf.json
if os.path.basename(os.getcwd())=='notebooks': os.chdir('..')
assert os.path.basename(os.getcwd())=='adamacs', ("Please move to the main directory")
import datajoint as dj; dj.conn()

[2023-05-08 14:15:16,093][INFO]: Connecting tobiasr@172.26.128.53:3306
[2023-05-08 14:15:16,139][INFO]: Connected tobiasr@172.26.128.53:3306


DataJoint connection (connected) tobiasr@172.26.128.53:3306

Manual entry:

### Imports and activation

Importing schema from `adamacs.pipeline` automatically activates relevant schema.

In [2]:
import datajoint as dj
from adamacs.pipeline import subject, session, surgery, scan, event, trial
from adamacs import utility
from adamacs.ingest import behavior as ibe
# check https://elements.datajoint.org/description/event/ for doc

DataJointError: Foreign key reference Scan could not be resolved

Assign easy names for relevant tables

In [ ]:
sub, lab, protocol, line, mutation, user, project, subject_genotype, subject_death = (
    subject.Subject(), subject.Lab(), subject.Protocol(), subject.Line(), 
    subject.Mutation(), subject.User(), subject.Project(), subject.SubjectGenotype(), 
    subject.SubjectDeath())

In [ ]:
event_types = ['main_track_gate', 'shutter', 'mini2p_frames', 'mini2p_lines', 'mini2p_volumes', 'aux_bpod_cam',
               'aux_bpod_visual', 'aux_bpod_reward', 'aux_bpod_tone']

for e in event_types:
    event.EventType.insert1({'event_type': e, 'event_type_description': ''}, skip_duplicates=True,)

In [ ]:
event.BehaviorRecording.drop()

In [ ]:
scan.Scan()

## Ingesting behavior

In [ ]:
ibe.ingest_aux("sess9FHS7Y22",verbose=True)

In [ ]:
event.Event()

In [ ]:
event.EventType()

In [ ]:
event.BehaviorRecording()

In [ ]:
dj.Diagram(subject.Subject) + dj.Diagram(event) + dj.Diagram(scan)

In [ ]:
from pywavesurfer import ws

In [ ]:
fp = event.BehaviorRecording.File().fetch('filepath')
print(fp)
curr_file = ws.loadDataFile(filename=fp[0], format_string='double' )


In [ ]:
event.BehaviorRecording.File()

In [ ]:
fp = event.BehaviorRecording.File().fetch('filepath')
print(fp)

In [ ]:
event.AlignmentEvent()

In [ ]:
trial.Trial()

In [ ]:
dj.Diagram(event)

In [ ]:
session.Session * session.SessionDirectory

In [ ]:
key='scan9FF6U114'
(scan.Scan & f'scan_id=\"{key}\"')

In [ ]:
scan.Scan()

In [ ]:
scan.ScanInfo.populate()

`CB DEV NOTE:` In demo file, there are no `scan.motor_position_at_zero` values. Set to 0 here to avoid downstream type errors when adding. Future would should set these to null when not present and revise downstream code.

In [ ]:
scan.ScanInfo()

In [ ]:
scan.ScanInfo.Field()

Note the relative path below:

In [ ]:
scan.ScanInfo.ScanFile()